In [12]:
import requests
import env
from requests.models import Request, Response
from env import Client_ID
import pandas as pd
from igdb.igdbapi_pb2 import GameResult
from typing import Dict, List, Optional, Union, cast


In [2]:
def get_api():   
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *;'
    r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)
    return access_token, r

In [9]:
Client_ID

'pmv2duet63rykzip3hbeb9bzopfe9y'

In [3]:
access_token, r = get_api()

In [6]:
access_token

'37atf9u217zzal38x2vtshq30cy9vh'

In [4]:
from igdb.wrapper import IGDBWrapper
wrapper = IGDBWrapper(f'{Client_ID}', f'Bearer {access_token}')

In [7]:
games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {0 * 409};')

HTTPError: 401 Client Error: Unauthorized for url: https://api.igdb.com/v4/games

In [5]:
testgames = []
for i in range(0, 409):
    games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 409};')
    testgames.append(games)

HTTPError: 401 Client Error: Unauthorized for url: https://api.igdb.com/v4/games

In [ ]:
len(testgames)

In [13]:
df = pd.read_csv('games.csv') 

/var/folders/nr/4dgzb75x0x5b5kl5610rvf340000gn/T/ipykernel_57878/3741917022.py:1: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('games.csv')


In [14]:
df.head()

,Unnamed: 0,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,...,aggregated_rating_count,multiplayer_modes,dlcs,ports,expansions,remakes,expanded_games,remasters,forks,standalone_expansions
0,0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40104,NaN,0,NaN,1498435200,"[20127, 1988884]",[5],Dogou Souken,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,85031,NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",[9],City Mysteries,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,99234,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]","[31, 32]",Sword of the Black Stone,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,148930,[33736],0,169077.0,1621959514,"[2052375, 2057300]","[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df.forks.isnull().sum()

204466

In [18]:
df.forks.notnull().sum()

34

In [22]:
df.dlcs.value_counts()

[144037]                                                                    2
[161755, 167297, 167298, 167299, 167300]                                    2
[27018, 27019, 27395, 75236]                                                2
[17473, 26282]                                                              2
[136063, 136064, 136065, 136187, 136188, 146158, 148173, 170286, 179296]    2
                                                                           ..
[170319]                                                                    1
[171336, 192242]                                                            1
[171582]                                                                    1
[167345, 174514]                                                            1
[121753, 121756]                                                            1
Name: dlcs, Length: 2168, dtype: int64

In [27]:
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.loc[df['franchises'].notnull()]

,Unnamed: 0,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,dlcs,ports,expansions,remakes,expanded_games,remasters,forks,standalone_expansions
34,34,193329,NaN,0,212201.0,1646480216,[2288508],"[5, 15, 25]",Lego Hero Factory: Brain Attack,NaN,"[750543, 750544, 750545]","[19164, 34024, 68855, 95340, 103298, 103301, 1...",lego-hero-factory-brain-attack,NaN,"[1, 268435461, 268435471, 268435481]",[1],1646700870,https://www.igdb.com/games/lego-hero-factory-b...,[65156],[268717],80f1eb3d-bf71-28de-8256-3e0a9a03fe94,NaN,NaN,NaN,NaN,"[165035, 165036]",[1],[3],NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3015.0,[51],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,47,88912,[74270],9,93117.0,1519382760,"[224818, 1999311, 2414235]","[8, 33]",Sonic the Hedgehog Classic,3192.0,"[206783, 206784, 206785, 206786, 206787, 20678...","[25901, 28070, 43367, 51485, 55042, 56033, 816...",sonic-the-hedgehog-classic,A remastered port of Sonic the Hedgehog for mo...,"[1, 268435464, 268435489, 536872420, 536882346...",[1],1654793072,https://www.igdb.com/games/sonic-the-hedgehog-...,NaN,"[76549, 139109, 139110, 275708]",f307126e-0e1f-d216-cc9a-91ac6b740667,1.368576e+09,"[34, 39]","[189309, 193673, 193674]",NaN,"[171549, 171550]",[1],[4],[89032],[881],"[1508, 11434, 17940, 24797]",NaN,80.000000,1.0,NaN,80.000000,1.0,NaN,NaN,NaN,2156.0,[457],NaN,[131033],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,54,6499,NaN,0,206656.0,1398719345,"[39078, 134506, 1921285]",[13],Lego Creator: Knights' Kingdom,NaN,"[726473, 726474, 726475]","[422, 3941, 10667, 19005, 61632, 66631, 70479,...",lego-creator-knights-kingdom,Lego Creator: Knights' Kingdom is a constructi...,"[268435469, 536871017, 536871304, 536871322, 5...",NaN,1648689886,https://www.igdb.com/games/lego-creator-knight...,NaN,NaN,fdeeeac9-1257-98ce-0556-ba31765d2d7e,9.782208e+08,[6],[15714],[3696],"[15446, 15447]",[1],"[1, 2]",NaN,NaN,"[105, 392, 410, 2152, 4142, 4161, 4181, 4651, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3015.0,[51],1.0,NaN,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,64,87751,NaN,0,NaN,1518912000,[1975797],[5],Counter-Strike Avalon,NaN,NaN,"[27270, 38030, 43367, 103281, 103292, 103298, ...",counter-strike-avalon,He is a good game,"[1, 268435461]",[1],1604620800,https://www.igdb.com/games/counter-strike-avalon,NaN,NaN,1346b187-115b-1624-29c7-e831ced48c2c,1.170979e+09,[8],[147874],"[21030, 21031]",[62394],"[2, 3, 5]",NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,[1216],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,69,56909,NaN,0,32199.0,1504529613,"[104099, 1925426]",[13],Kamen Rider Atsume,NaN,NaN,"[17130, 18623, 26950, 31515, 33153, 36346, 448...",kamen-rider-atsume,Hit-Point's follow-up to Neko Atsume has playe...,"[27, 268435469, 536870926, 536871262, 53687206...",[27],1653979388,https://www.igdb.com/games/kamen-rider-atsume,NaN,"[286560, 286561, 286562]",054e7210-e78b-7f3b-cf06-64aaeeabd8ae,1.459123e+09,"[34, 39]","[361900, 361909]",NaN,NaN,[1],[4],[89967],NaN,"[14, 350, 1148, 1682, 2387, 4134, 4142, 4377, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3146.0,[696],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204487,487,136539,"[9371, 9372, 9373]",1,159599.0,1596977606,"[1997165, 1997205, 1997237, 1997470, 2018308, ...","[5, 33]",DOOM Eternal: T

In [30]:
df.columns

Index(['Unnamed: 0', 'id', 'artworks', 'category', 'cover', 'created_at', 'external_games', 'genres', 'name', 'parent_game', 'screenshots', 'similar_games', 'slug', 'summary', 'tags', 'themes', 'updated_at', 'url', 'videos', 'websites', 'checksum', 'first_release_date', 'platforms', 'release_dates', 'age_ratings', 'involved_companies', 'game_modes', 'player_perspectives', 'alternative_names', 'game_engines', 'keywords', 'status', 'rating', 'rating_count', 'storyline', 'total_rating', 'total_rating_count', 'version_parent', 'version_title', 'hypes', 'collection', 'franchises', 'follows', 'bundles', 'franchise', 'aggregated_rating', 'aggregated_rating_count', 'multiplayer_modes', 'dlcs', 'ports', 'expansions', 'remakes', 'expanded_games', 'remasters', 'forks', 'standalone_expansions'], dtype='object')

In [28]:
df.shape

(204500, 56)

In [31]:
df = df.drop(columns=['artworks', 'cover','screenshots', 'videos', 'checksum', 'alternative_names', 'rating_count', 'total_rating_count', 'version_parent', 'hypes', 'franchises', 'follows', 'aggregated_rating_count'])
df.shape

(204500, 43)